In [1]:
pip install opendatasets

In [2]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/jessicali9530/lfw-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: adityapathak03
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/jessicali9530/lfw-dataset


100%|██████████| 112M/112M [00:00<00:00, 125MB/s]


In [3]:
!pip install facenet-pytorch faiss-cpu pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0

In [1]:
import os
import pickle
import numpy as np
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1
import faiss
import torch

In [9]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(image_size = 160, margin=0, min_face_size = 40)#, device = 'device')
facenet = InceptionResnetV1(pretrained='vggface2').eval()#.to(device)

In [10]:
# Preprocessing image
def preprocess_image(image_path):
    img = Image.open(image_path).convert('RGB')
    face = mtcnn(img)
    if face is None:
        raise ValueError(f"No face detected in {image_path}")
    return face.unsqueeze(0)#.to(device)

# Generating embedding
def generate_embedding(image_path):
    face_tensor = preprocess_image(image_path)
    with torch.no_grad():
        embedding = facenet(face_tensor).cpu().numpy()
    return embedding

# Creating FAISS index
def create_faiss_index(dataset_dir):
    embeddings = []
    file_paths = []

    for person_folder in os.listdir(dataset_dir):
        folder_path = os.path.join(dataset_dir, person_folder)
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            try:
                emb = generate_embedding(image_path)
                embeddings.append(emb.flatten())
                file_paths.append(image_path)
            except ValueError as e:
                print(e)

    embeddings = np.vstack(embeddings)
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)

    return index, file_paths

In [11]:
dataset_dir = "/content/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled" # Dataset path

# Creating and saving the FAISS index
index, file_paths = create_faiss_index(dataset_dir)

# Saving the index and file paths using pickle
with open("faiss_model.pkl", "wb") as f:
    pickle.dump((index, file_paths), f)

print("FAISS index and file paths saved successfully.")

No face detected in /content/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled/Marilyn_Monroe/Marilyn_Monroe_0001.jpg
FAISS index and file paths saved successfully.
